In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
#   Convert RGB images to CIE-LAB Color File

from __future__ import print_function
import argparse
import os

import numpy as np
import pickle
import random
from keras.preprocessing.image import list_pictures, array_to_img
from image_ext import list_pictures_in_multidir, load_imgs_asarray, img_dice_coeff

np.random.seed(2016)

/home/nakazawa_atsushi/anaconda3/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/nakazawa_atsushi/anaconda3/envs/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
def load_fnames(paths):
    f = open(paths)
    data1 = f.read()
    f.close()
    lines = data1.split('\n')
    #print(len(lines))
    # 最終行は空行なので消す
    del(lines[len(lines)-1])
    #print(len(lines))
    return lines

In [4]:
def make_fnames(fnames,fpath,fpath_mask,mask_ext):
    fnames_img = [];
    fnames_mask= [];
    
    for i in range(len(fnames)):
        fnames_img.append(fpath + '/' + fnames[i]);
        fnames_mask.append(fpath_mask + '/' + mask_ext + fnames[i]);
        
    return [fnames_img,fnames_mask]

In [5]:
def RGB2RGChroma(img):
    # convert RGB to RG Chromaticity Image
    # https://en.wikipedia.org/wiki/Rg_chromaticity
    ret = np.zeros((img.shape[0],img.shape[1],img.shape[2]),'float')
    
    R = img[:,:,0]
    G = img[:,:,1]
    B = img[:,:,2]
    I = R + G + B + 1
    r = (R+1)/I
    g = (G+1)/I
    b = (B+1)/I
    R2 = r*G / g
    B2 = ((1 - r - g)*G) / g
    
    ret[:,:,0] = r
    ret[:,:,1] = g
    ret[:,:,2] = 0
    
    return ret

In [6]:
def LabConversion(img):
    ret = cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2LAB)
    return ret

In [7]:
def LabAugumentation(img,L,a,b):
    ret = cv2.cvtColor(img.astype('uint8'), cv2.COLOR_BGR2LAB)
    # print(ret)
    ret[:,:,0] = ret[:,:,0] + L
    ret[:,:,1] = ret[:,:,1] + a
    ret[:,:,2] = ret[:,:,2] + b
    ret = np.clip(ret,0,255)
    ret = cv2.cvtColor(ret, cv2.COLOR_LAB2BGR)    
    return ret

In [8]:
#
#  MAIN STARTS FROM HERE
#
if __name__ == '__main__':
    
    target_size = (224, 224)
    dim_ordering = 'channels_last'
    
#    fnames = load_fnames('data.nnlab/list_files_all.txt')
#    [fpaths_xs_train,fpaths_ys_train] = make_fnames(fnames,'data.nnlab/image','data.nnlab/gt','')
#    [fpaths_xs_out,fpaths_ys_out] = make_fnames(fnames,'data.nnlab.LAB/img','data.nnlab/gt','')
    fnames = load_fnames('data/list_files_all.txt')
    [fpaths_xs_train,fpaths_ys_train] = make_fnames(fnames,'data/img','data/mask','OperatorA_')
    [fpaths_xs_out,fpaths_ys_out] = make_fnames(fnames,'data_augumented/img','data_augumented/mask','OperatorA_')

    if os.path.exists('data_augumented') == 0:
        os.mkdir('data_augumented')
        os.mkdir('data_augumented/img')
        os.mkdir('data_augumented/mask')
        
#    X_img = load_imgs_asarray(fpaths_xs_train, grayscale=False, target_size=target_size,
#                                dim_ordering=dim_ordering)
#    Y_img = load_imgs_asarray(fpaths_ys_train, grayscale=True, target_size=target_size,
#                                dim_ordering=dim_ordering)    

In [9]:
    import matplotlib.pyplot as plt
    from PIL import Image 
    import cv2
    
    aug_div = [20,30,30]    
    
    fpath_img = 'data_augumented/img'
    fpath_mask = 'data_augumented/mask'
    
    for i in range(len(fpaths_xs_train)):
        img = cv2.imread(fpaths_xs_train[i])
        mask = cv2.imread(fpaths_ys_train[i])
        
        fnames_img = fpath_img + '/' + fnames[i];
        cv2.imwrite(fnames_img, img)
        
        fnames_mask = fpath_mask + '/' + fnames[i];
        print(fnames_mask)
        cv2.imwrite(fnames_mask,mask)
        
        for j in range(3):
            aug_param = [ random.randint(-aug_div[0],aug_div[0]), random.randint(-aug_div[1],aug_div[1]),
                             random.randint(-aug_div[2],aug_div[2]) ]
            print(aug_param)
            im = LabAugumentation(img,aug_param[0],aug_param[1],aug_param[2])
            
            fnames_img = '%s/%03d_%s'%(fpath_img, j, fnames[i])
            fnames_mask = '%s/%03d_%s'%(fpath_mask, j, fnames[i])
            print(fnames_img)
            print(fnames_mask)
            cv2.imwrite(fnames_img, im)
            cv2.imwrite(fnames_mask, mask)
        

data_augumented/mask/C100_S1_I1.tiff
[13, -22, -5]
data_augumented/img/000_C100_S1_I1.tiff
data_augumented/mask/000_C100_S1_I1.tiff
[14, 5, 28]
data_augumented/img/001_C100_S1_I1.tiff
data_augumented/mask/001_C100_S1_I1.tiff
[1, 1, -2]
data_augumented/img/002_C100_S1_I1.tiff
data_augumented/mask/002_C100_S1_I1.tiff
data_augumented/mask/C100_S1_I10.tiff
[4, 1, 0]
data_augumented/img/000_C100_S1_I10.tiff
data_augumented/mask/000_C100_S1_I10.tiff
[13, -14, 9]
data_augumented/img/001_C100_S1_I10.tiff
data_augumented/mask/001_C100_S1_I10.tiff
[13, 26, -5]
data_augumented/img/002_C100_S1_I10.tiff
data_augumented/mask/002_C100_S1_I10.tiff
data_augumented/mask/C100_S1_I11.tiff
[15, -1, 30]
data_augumented/img/000_C100_S1_I11.tiff
data_augumented/mask/000_C100_S1_I11.tiff
[17, 2, -7]
data_augumented/img/001_C100_S1_I11.tiff
data_augumented/mask/001_C100_S1_I11.tiff
[-15, -1, 6]
data_augumented/img/002_C100_S1_I11.tiff
data_augumented/mask/002_C100_S1_I11.tiff
data_augumented/mask/C100_S1_I12.ti

[14, -14, -16]
data_augumented/img/001_C100_S2_I5.tiff
data_augumented/mask/001_C100_S2_I5.tiff
[20, -12, 2]
data_augumented/img/002_C100_S2_I5.tiff
data_augumented/mask/002_C100_S2_I5.tiff
data_augumented/mask/C100_S2_I6.tiff
[4, -30, -22]
data_augumented/img/000_C100_S2_I6.tiff
data_augumented/mask/000_C100_S2_I6.tiff
[-11, -22, 18]
data_augumented/img/001_C100_S2_I6.tiff
data_augumented/mask/001_C100_S2_I6.tiff
[2, -17, 21]
data_augumented/img/002_C100_S2_I6.tiff
data_augumented/mask/002_C100_S2_I6.tiff
data_augumented/mask/C100_S2_I7.tiff
[14, -10, 18]
data_augumented/img/000_C100_S2_I7.tiff
data_augumented/mask/000_C100_S2_I7.tiff
[7, -13, -16]
data_augumented/img/001_C100_S2_I7.tiff
data_augumented/mask/001_C100_S2_I7.tiff
[-2, -11, 3]
data_augumented/img/002_C100_S2_I7.tiff
data_augumented/mask/002_C100_S2_I7.tiff
data_augumented/mask/C100_S2_I8.tiff
[8, -5, 22]
data_augumented/img/000_C100_S2_I8.tiff
data_augumented/mask/000_C100_S2_I8.tiff
[4, -21, 21]
data_augumented/img/001_

data_augumented/mask/C101_S2_I2.tiff
[-8, 2, 16]
data_augumented/img/000_C101_S2_I2.tiff
data_augumented/mask/000_C101_S2_I2.tiff
[12, 30, -15]
data_augumented/img/001_C101_S2_I2.tiff
data_augumented/mask/001_C101_S2_I2.tiff
[-15, 1, -30]
data_augumented/img/002_C101_S2_I2.tiff
data_augumented/mask/002_C101_S2_I2.tiff
data_augumented/mask/C101_S2_I3.tiff
[-19, 2, -1]
data_augumented/img/000_C101_S2_I3.tiff
data_augumented/mask/000_C101_S2_I3.tiff
[-8, -19, -9]
data_augumented/img/001_C101_S2_I3.tiff
data_augumented/mask/001_C101_S2_I3.tiff
[16, 26, 28]
data_augumented/img/002_C101_S2_I3.tiff
data_augumented/mask/002_C101_S2_I3.tiff
data_augumented/mask/C101_S2_I4.tiff
[-18, -25, -17]
data_augumented/img/000_C101_S2_I4.tiff
data_augumented/mask/000_C101_S2_I4.tiff
[-5, 24, -22]
data_augumented/img/001_C101_S2_I4.tiff
data_augumented/mask/001_C101_S2_I4.tiff
[3, 23, 8]
data_augumented/img/002_C101_S2_I4.tiff
data_augumented/mask/002_C101_S2_I4.tiff
data_augumented/mask/C101_S2_I5.tiff
[3

[-2, 0, -20]
data_augumented/img/001_C102_S2_I12.tiff
data_augumented/mask/001_C102_S2_I12.tiff
[19, -10, -25]
data_augumented/img/002_C102_S2_I12.tiff
data_augumented/mask/002_C102_S2_I12.tiff
data_augumented/mask/C102_S2_I13.tiff
[14, -5, 9]
data_augumented/img/000_C102_S2_I13.tiff
data_augumented/mask/000_C102_S2_I13.tiff
[19, -22, 22]
data_augumented/img/001_C102_S2_I13.tiff
data_augumented/mask/001_C102_S2_I13.tiff
[-1, 7, 20]
data_augumented/img/002_C102_S2_I13.tiff
data_augumented/mask/002_C102_S2_I13.tiff
data_augumented/mask/C102_S2_I14.tiff
[9, 28, 2]
data_augumented/img/000_C102_S2_I14.tiff
data_augumented/mask/000_C102_S2_I14.tiff
[18, -20, 19]
data_augumented/img/001_C102_S2_I14.tiff
data_augumented/mask/001_C102_S2_I14.tiff
[-14, -7, -2]
data_augumented/img/002_C102_S2_I14.tiff
data_augumented/mask/002_C102_S2_I14.tiff
data_augumented/mask/C102_S2_I15.tiff
[-9, 16, -7]
data_augumented/img/000_C102_S2_I15.tiff
data_augumented/mask/000_C102_S2_I15.tiff
[-2, -26, 5]
data_aug

data_augumented/img/000_C103_S2_I11.tiff
data_augumented/mask/000_C103_S2_I11.tiff
[13, 4, 25]
data_augumented/img/001_C103_S2_I11.tiff
data_augumented/mask/001_C103_S2_I11.tiff
[15, -14, 8]
data_augumented/img/002_C103_S2_I11.tiff
data_augumented/mask/002_C103_S2_I11.tiff
data_augumented/mask/C103_S2_I12.tiff
[2, 25, 24]
data_augumented/img/000_C103_S2_I12.tiff
data_augumented/mask/000_C103_S2_I12.tiff
[13, 11, 13]
data_augumented/img/001_C103_S2_I12.tiff
data_augumented/mask/001_C103_S2_I12.tiff
[18, -27, 16]
data_augumented/img/002_C103_S2_I12.tiff
data_augumented/mask/002_C103_S2_I12.tiff
data_augumented/mask/C103_S2_I13.tiff
[3, 28, -17]
data_augumented/img/000_C103_S2_I13.tiff
data_augumented/mask/000_C103_S2_I13.tiff
[3, -27, 1]
data_augumented/img/001_C103_S2_I13.tiff
data_augumented/mask/001_C103_S2_I13.tiff
[2, -14, -12]
data_augumented/img/002_C103_S2_I13.tiff
data_augumented/mask/002_C103_S2_I13.tiff
data_augumented/mask/C103_S2_I14.tiff
[5, -12, -20]
data_augumented/img/00

...

data_augumented/mask/C96_S1_I4.tiff
[-3, 0, 16]
data_augumented/img/000_C96_S1_I4.tiff
data_augumented/mask/000_C96_S1_I4.tiff
[20, 28, 9]
data_augumented/img/001_C96_S1_I4.tiff
data_augumented/mask/001_C96_S1_I4.tiff
[-4, -9, 0]
data_augumented/img/002_C96_S1_I4.tiff
data_augumented/mask/002_C96_S1_I4.tiff
data_augumented/mask/C96_S1_I5.tiff
[-3, -27, -7]
data_augumented/img/000_C96_S1_I5.tiff
data_augumented/mask/000_C96_S1_I5.tiff
[15, 30, -5]
data_augumented/img/001_C96_S1_I5.tiff
data_augumented/mask/001_C96_S1_I5.tiff
[11, -25, 17]
data_augumented/img/002_C96_S1_I5.tiff
data_augumented/mask/002_C96_S1_I5.tiff
data_augumented/mask/C96_S1_I6.tiff
[1, -10, -10]
data_augumented/img/000_C96_S1_I6.tiff
data_augumented/mask/000_C96_S1_I6.tiff
[-9, -21, 0]
data_augumented/img/001_C96_S1_I6.tiff
data_augumented/mask/001_C96_S1_I6.tiff
[15, -28, -7]
data_augumented/img/002_C96_S1_I6.tiff
data_augumented/mask/002_C96_S1_I6.tiff
data_augumented/mask/C96_S1_I7.tiff
[19, 0, 9]
data_augumented/

data_augumented/mask/C97_S1_I2.tiff
[-9, 5, 18]
data_augumented/img/000_C97_S1_I2.tiff
data_augumented/mask/000_C97_S1_I2.tiff
[-2, -25, -29]
data_augumented/img/001_C97_S1_I2.tiff
data_augumented/mask/001_C97_S1_I2.tiff
[-18, 4, -3]
data_augumented/img/002_C97_S1_I2.tiff
data_augumented/mask/002_C97_S1_I2.tiff
data_augumented/mask/C97_S1_I3.tiff
[1, 27, -16]
data_augumented/img/000_C97_S1_I3.tiff
data_augumented/mask/000_C97_S1_I3.tiff
[1, 25, 23]
data_augumented/img/001_C97_S1_I3.tiff
data_augumented/mask/001_C97_S1_I3.tiff
[10, -10, -4]
data_augumented/img/002_C97_S1_I3.tiff
data_augumented/mask/002_C97_S1_I3.tiff
data_augumented/mask/C97_S1_I4.tiff
[10, 14, -18]
data_augumented/img/000_C97_S1_I4.tiff
data_augumented/mask/000_C97_S1_I4.tiff
[20, -23, 9]
data_augumented/img/001_C97_S1_I4.tiff
data_augumented/mask/001_C97_S1_I4.tiff
[15, -10, -16]
data_augumented/img/002_C97_S1_I4.tiff
data_augumented/mask/002_C97_S1_I4.tiff
data_augumented/mask/C97_S1_I5.tiff
[-9, -27, 25]
data_augum

[7, -30, 18]
data_augumented/img/001_C99_S1_I13.tiff
data_augumented/mask/001_C99_S1_I13.tiff
[11, -29, 30]
data_augumented/img/002_C99_S1_I13.tiff
data_augumented/mask/002_C99_S1_I13.tiff
data_augumented/mask/C99_S1_I14.tiff
[-4, -2, -10]
data_augumented/img/000_C99_S1_I14.tiff
data_augumented/mask/000_C99_S1_I14.tiff
[-14, 4, 18]
data_augumented/img/001_C99_S1_I14.tiff
data_augumented/mask/001_C99_S1_I14.tiff
[17, 12, -9]
data_augumented/img/002_C99_S1_I14.tiff
data_augumented/mask/002_C99_S1_I14.tiff
data_augumented/mask/C99_S1_I15.tiff
[1, 23, -11]
data_augumented/img/000_C99_S1_I15.tiff
data_augumented/mask/000_C99_S1_I15.tiff
[-13, -24, 27]
data_augumented/img/001_C99_S1_I15.tiff
data_augumented/mask/001_C99_S1_I15.tiff
[7, 24, -19]
data_augumented/img/002_C99_S1_I15.tiff
data_augumented/mask/002_C99_S1_I15.tiff
data_augumented/mask/C99_S1_I2.tiff
[-10, -17, -29]
data_augumented/img/000_C99_S1_I2.tiff
data_augumented/mask/000_C99_S1_I2.tiff
[5, -11, 0]
data_augumented/img/001_C99

[-10, 17, -13]
data_augumented/img/001_C9_S1_I1.tiff
data_augumented/mask/001_C9_S1_I1.tiff
[19, 23, 2]
data_augumented/img/002_C9_S1_I1.tiff
data_augumented/mask/002_C9_S1_I1.tiff
data_augumented/mask/C9_S1_I10.tiff
[9, -6, 28]
data_augumented/img/000_C9_S1_I10.tiff
data_augumented/mask/000_C9_S1_I10.tiff
[9, -16, 22]
data_augumented/img/001_C9_S1_I10.tiff
data_augumented/mask/001_C9_S1_I10.tiff
[-12, -3, 28]
data_augumented/img/002_C9_S1_I10.tiff
data_augumented/mask/002_C9_S1_I10.tiff
data_augumented/mask/C9_S1_I11.tiff
[1, -29, -25]
data_augumented/img/000_C9_S1_I11.tiff
data_augumented/mask/000_C9_S1_I11.tiff
[14, -7, -13]
data_augumented/img/001_C9_S1_I11.tiff
data_augumented/mask/001_C9_S1_I11.tiff
[18, 23, -25]
data_augumented/img/002_C9_S1_I11.tiff
data_augumented/mask/002_C9_S1_I11.tiff
data_augumented/mask/C9_S1_I12.tiff
[-4, -4, -20]
data_augumented/img/000_C9_S1_I12.tiff
data_augumented/mask/000_C9_S1_I12.tiff
[-3, 13, -11]
data_augumented/img/001_C9_S1_I12.tiff
data_augum

[8, -28, 13]
data_augumented/img/002_C9_S2_I7.tiff
data_augumented/mask/002_C9_S2_I7.tiff
data_augumented/mask/C9_S2_I8.tiff
[-2, 24, 28]
data_augumented/img/000_C9_S2_I8.tiff
data_augumented/mask/000_C9_S2_I8.tiff
[-8, -22, 30]
data_augumented/img/001_C9_S2_I8.tiff
data_augumented/mask/001_C9_S2_I8.tiff
[17, 20, 19]
data_augumented/img/002_C9_S2_I8.tiff
data_augumented/mask/002_C9_S2_I8.tiff
data_augumented/mask/C9_S2_I9.tiff
[2, -22, -7]
data_augumented/img/000_C9_S2_I9.tiff
data_augumented/mask/000_C9_S2_I9.tiff
[-19, -10, -27]
data_augumented/img/001_C9_S2_I9.tiff
data_augumented/mask/001_C9_S2_I9.tiff
[4, -23, 4]
data_augumented/img/002_C9_S2_I9.tiff
data_augumented/mask/002_C9_S2_I9.tiff
